## 安装依赖

如果当前环境已经有相关依赖了，则不用执行

In [ ]:
! pip install datasets==3.6.0 evaluate transformers rouge-score nltk

## 加载数据

本案例使用的是XSum数据集，其中包含BBC的文章以及单句的摘要等信息；数据集有点大，在本地下载时请注意磁盘空间；

In [ ]:
from datasets import load_dataset
from evaluate import load

model_checkpoint = "t5-small"

raw_datasets = load_dataset("xsum")
metric = load("rouge")

你也可以看看datasets的数据结构长什么样：

In [ ]:
print(raw_datasets)

```json
DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})
```

可以看到训练集的数据有20w条；

其中：document是文章，summary是对应的摘要，id即为数据id；

## 数据集可视化

为了能够进一步理解数据长什么样子，下面的函数将从数据集里随机选择几个例子进行展示；

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(raw_datasets["train"])

## 数据预处理

在将这些数据样本输入模型之前，需要对样本进行预处理，使用分词器Tokenizer对样本进行编码（即将分词后的token转成词汇表中，每一个词对应的ID），然后转换为模型所需要的格式，即可；

In [ ]:
# 加载分词器
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

如果你想看到分词后的句子是什么样子的话

In [ ]:
tokenizer("Hello, this one sentence")

```json
{'input_ids': [8774, 6, 48, 80, 7142, 1], 'attention_mask': [1, 1, 1, 1, 1, 1]}
```

如果你使用的是T5系列模型，那么在做微调时，需要在样本头添加一个前缀，告诉LLM你在做什么任务；本案例是摘要生成，那么就加一个summarize即可；

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
## 封装数据预处理函数

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    # 添加样本前缀
    inputs = [prefix + doc for doc in examples["document"]]
    # 当样本长度超过max_length，由于truncation=True，所以会自动发生截断
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 在seq2seq模型中，source text是用于encoder，target text是用于decoder在训练时需要预测的标签
    # 所以这里会选择把summary转成input_ids组成的labels，用于decoder预测
    # 如果只是简单的tokenizer(examples["summary"])，分词器会认为该样本只是普通数据，不知道这是训练的目标；
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

preprocess_function(raw_datasets['train'][:2])

## 数据映射

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

## 加载预训练模型

In [ ]:
# 加载预训练模型
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## 封装训练参数配置器

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    eval_strategy = "epoch", # 每个epoch训练完即评估
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3, # seq2seqTrainer会定期保存模型权重，所以设置最多保存3个
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

## 获取数据加载器

它会将每个batch的input进行最大长度对齐，同样也会将labels填充到最大长度；

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

## 封装评估函数

In [ ]:
# 封装评估指标计算函数
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # 把labels中-100的值，替换成分词器的特殊token，pad_token，在decode时可以跳过
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE 指标库希望输入的参考答案和预测答案是 按句子分行的文本。
    # nltk.sent_tokenize：会把一句话切分成多个句子。例如："I love NLP. It is fun." → ["I love NLP.", "It is fun."]
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # use_stemmer=True：主要是把单词提取词干，例如：running → run，避免因为词形变化导致分数偏低。
    # use_aggregator=True：会对整个 dataset 的结果聚合（平均），而不是逐个样本输出
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # 把结果转成百分制
    result = {key: value * 100 for key, value in result.items()}

    # 统计predictions里面非特殊token的数量，再取平均值
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## 构建训练器

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## 开始训练

In [ ]:
# 一个epoch需要跑2个半小时

trainer.train()